In [9]:
import os
from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib as plt
%matplotlib inline

In [10]:
def resize(img):
    w, h = img.size
    BASE_WIDTH, BASE_HEIGHT = 128, 32

    def adjust_dim():
        if BASE_HEIGHT >= h * BASE_WIDTH / w:
            return BASE_WIDTH, h * BASE_WIDTH / w
        elif BASE_WIDTH >= w * BASE_HEIGHT / h:
            return w * BASE_HEIGHT / h, BASE_HEIGHT
        else:
            return BASE_WIDTH, BASE_HEIGHT

    new_dim = tuple(map(int, adjust_dim()))
    blank_img = Image.new('L', (BASE_WIDTH, BASE_HEIGHT), 255)
    img = img.resize(new_dim, Image.ANTIALIAS)
    assert(img.size[0] <= blank_img.size[0] and img.size[1] <= blank_img.size[1])
    blank_img.paste(img)
    return blank_img

In [11]:
def image_loader(filepath):
    X = []
    X_filename = []
    for dir, parent_file, files in os.walk(filepath):
        for filename in files:
            path = os.path.join(dir, filename)
            if filename[-3 : ] == 'png' and os.stat(path).st_size > 0:
                img = resize(Image.open(path).copy())
                X.append(np.array(img))
                X_filename.append(filename[:-4])
    return np.array(X), np.array(X_filename)

In [12]:
char_list = ''' !"#&'()*+,-./0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz'''

In [13]:
def encode_to_labels(word):
    dig_lst = []
    for index, char in enumerate(word):
        dig_lst.append(char_list.index(char))
    return np.array(dig_lst)

In [14]:
def text_loader(filepath):
    Y = {}
    file = open(filepath, 'r')
    for info in file:
        if info[0] == '#':
            pass
        info = info.split()
        filename = info[0]
        word = info[-1]	
        Y[filename] = word
        
    return Y

In [2]:
def data_loader(image_filepath, text_filepath):
    X, X_filename = image_loader(image_filepath)
    X, X_filename = X[:100000], X_filename[:100000]
    Y_dict = text_loader(text_filepath)
    split = int(0.95 * len(X))
    
    X_train = X[:split]
    X_test = X[split:]
    X_filename_train = X_filename[:split]
    X_filename_test = X_filename[split:]
    
    Y_train, Y_test, Y_train_encoded, Y_test_encoded = [], [], [], []
    
    
    for i in range(len(X_filename_train)):
        Y_train.append(Y_dict[X_filename_train[i]])
        Y_train_encoded.append(encode_to_labels(Y_dict[X_filename_train[i]]))
    for i in range (len(X_filename_test)):
        Y_test.append(Y_dict[X_filename_test[i]])
        Y_test_encoded.append(encode_to_labels(Y_dict[X_filename_test[i]]))
    
    Y_train_encoded = pad_sequences(Y_train_encoded, maxlen = 32, padding ='post', value = len(char_list))
    Y_test_encoded = pad_sequences(Y_test_encoded, maxlen = 32, padding ='post', value = len(char_list))

    return X_train, X_test, Y_train, Y_test, Y_train_encoded, Y_test_encoded 